EDA guide: https://miykael.github.io/blog/2022/advanced_eda/

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from scipy import stats
import numpy as np
import seaborn as sns
import xgboost as xgb
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import RFE
# from tsfresh import extract_features

In [ ]:
train_targets = pd.read_parquet('./data/A/train_targets.parquet')
X_test_estimated = pd.read_parquet('./data/A/X_test_estimated.parquet')
X_train_estimated = pd.read_parquet('./data/A/X_train_estimated.parquet')
X_train_observed = pd.read_parquet('./data/A/X_train_observed.parquet')

In [ ]:
X_train_observed.describe()

In [ ]:
X_train_estimated.describe()

In [ ]:
train_targets.describe()

From these to descriptions we can see that X_train_observed stops where X_train_estimated start. This gives us an intuition to concat the two datasets.

We also see that the date_forecast feature contains values for each quarter of an hour. This needs to be fixed since train_targets only contain values for each hour.

In [ ]:
df = pd.concat([X_train_observed, X_train_estimated])

In [ ]:
df = pd.merge(df, train_targets, left_on='date_forecast', right_on='time', how='inner')

Merging the train_targets into df to make it easier with analysis later on. This also removed each value not containing a whole hour.

# 1. Datatypes and structure

In [ ]:
pd.value_counts(df.dtypes)

This shows that we have three datetime columns. Let's further explore what these are:

## 1.1 Non-numeric features

In [ ]:
df.select_dtypes(exclude='number').head()

From this we can extract two major points of information: 
- First of all, we have 24hrs of data for just about each day in the dataset. 
- Secondly, after merging the X_train_observed, X_train_estimated and train_targets we now have these three datetime columns. From this we see: date_calc contain Nat values, date_forecast and time has the same values. These findings indicate that we can remove either time or date_forecast and have to look deeper into date_calc.

In [ ]:
df['date_calc'].describe()

When describing the feature we can see that it has values for almost a year. This may indicate that something went wrong when either concating or merging the datasets. Let's investigate:

In [ ]:
X_train_estimated[['date_forecast', 'date_calc']].head()

From this we can see that date_calc indicates which day a forecast was calculated. Based on this, we can for now safely remove it.

In [ ]:
df = df.drop(columns=['date_calc', 'date_forecast'])
df.select_dtypes(exclude='number').head()

In [ ]:
df.describe(exclude='number')

We have a total of 34.061 rows of data, ranging from the mid of June 2019 to the mid of April 2023

## 1.2 Numeric features

In [ ]:
unique_values = df.select_dtypes(exclude='datetime').nunique().sort_values()
unique_values.plot.bar(logy=True, figsize=(15, 4))

From this we can see that most features are continuous, but three of the features are single valued. This means that they only have one value across all rows. This is something we need to investigate further:

In [ ]:
df[['elevation:m', 'snow_density:kgm3', 'snow_drift:idx']].head(20)

These features have major issues and can therefor be removed from the dataset

In [ ]:
df = df.drop(columns=['elevation:m', 'snow_density:kgm3', 'snow_drift:idx'], axis=1)
pd.value_counts(df.dtypes)

As we can see, we now have three less float32 features, meaning that the features was properly removed.

# 2. Quality check

In [ ]:
df.isna().sum()

We have a total of 7777 missing values in ceiling_height_agl:m and 3063 missing values in cloud_base_agl:m. There are too many missing values to remove the rows containing them, and we don't know the importance of the features yet, so we can't just remove them. Our only choice is then to impute them. A normal strategy for numeric features is using the mean of this feature:

In [ ]:
imputer = SimpleImputer(strategy='mean')
df[['ceiling_height_agl:m', 'cloud_base_agl:m']] = imputer.fit_transform(df[['ceiling_height_agl:m', 'cloud_base_agl:m']])

In [ ]:
df.isna().sum()

All the null values is now removed

# REMEMBER TO FIND OUTLIERS

# 3. Shallow Feature Information
Considering the dimensionality of the dataset, analyzing each feature independently is close to impossible. Therefor, we will first plot each feature and only comment those with some interesting values.

## 3.1 Feature Distributions

In [ ]:
# Drop the index and target variable columns for feature distribution analysis
X_observed = X_train_observed.drop(columns=['date_calc', 'date_forecast'])
X_estimated = X_train_estimated.drop(columns=['date_calc', 'date_forecast'])
def plot_grid_feature_distributions(observed_df, estimated_df):
    features = observed_df.columns
    num_features = len(features)
    num_rows = -(-num_features // 5)  # Calculate rows needed, rounding up
    
    fig, axes = plt.subplots(num_rows, 5, figsize=(20, num_rows * 4))
    axes = axes.flatten()  # Flatten the 2D array to 1D
    
    for i, feature in enumerate(features):
        ax = axes[i]
        sns.histplot(observed_df[feature], kde=False, bins=50, color='b', label='Observed', stat='probability', ax=ax, element='step')
        sns.histplot(estimated_df[feature], kde=False, bins=50, color='r', label='Estimated', stat='probability', ax=ax, element='step')
        ax.set_title(f'{feature}')
        ax.set_xlabel('')
        ax.set_ylabel('Normalized Count')
        ax.legend()
    
    # Remove extra subplots
    for i in range(num_features, num_rows * 5):
        fig.delaxes(axes[i])
    
    plt.tight_layout()
    plt.show()

# Call the function to plot the grid
plot_grid_feature_distributions(X_observed, X_estimated)

absolute_humidity_2m:gm3: Shows a bimodal distribution, indicating two distinct groups or conditions in the dataset.

air_density_2m:kgm3: Almost normally distributed but has some outliers on the lower end.

ceiling_height_agl:m: Highly skewed to the right, meaning most of the values are clustered at the lower end.

clear_sky_energy_1h:J: Most values are zero, but there are some with higher values, indicating specific conditions where clear sky energy is non-zero.

total_cloud_cover:p: The distribution is almost binary, with most values at either 0 or 100, indicating clear sky or full cloud cover.

visibility:m: This feature also shows a bimodal distribution, indicating two different visibility conditions.

pv_measurement: Highly skewed towards the left. Indicating, most of the time, the solar power consumption is very low.

## 3.2 Linear Correlations

Highly Interdependent Features:
pressure_50m:hPa, sfc_pressure:hPa, and pressure_100m:hPa: These features have correlations very close to 1, indicating they are almost identical. One or two of these could likely be removed without losing much information.

dew_point_2m:K and absolute_humidity_2m:gm3: With a correlation of approximately 0.97, these variables are highly interrelated, suggesting redundancy.

clear_sky_energy_1h:J and clear_sky_rad:W: These also have a very high correlation of approximately 0.99, indicating potential redundancy.

Weather and Time-related Features:
fresh_snow_24h:cm, fresh_snow_12h:cm, fresh_snow_6h:cm, fresh_snow_3h:cm, and fresh_snow_1h:cm: These features are highly correlated with each other, ranging from 0.78 to 0.95, indicating they carry similar information about snowfall over different time periods.

solar_zenith_angle:d and sun_elevation:d: These features have a high negative correlation of -0.99, which makes sense because as the sun rises, the solar zenith angle decreases.

Correlations with Target Variable (pv_measurement):
clear_sky_energy_1h:J: This feature has a high positive correlation of 0.78 with the target variable 'pv_measurement', signifying its importance in predicting the target.

sun_elevation:d: This feature also shows a high positive correlation of 0.76 with the target variable, indicating its relevance in predicting solar energy production.

Other Interesting Correlations:
air_density_2m:kgm3 and t_1000hPa:K: These features have a high negative correlation of -0.91, which is logical given the inverse relationship between air density and temperature.

## 3.3 Non-linear Correlations

Highly Interdependent Features:
pressure_50m:hPa, sfc_pressure:hPa, and pressure_100m:hPa: These features have near-perfect Spearman correlations, suggesting they convey almost identical information. Consider dropping some to remove redundancy.

dew_point_2m:K and absolute_humidity_2m:gm3: With a Spearman correlation of around 0.97, these features are highly associated and possibly redundant.

clear_sky_energy_1h:J and clear_sky_rad:W: These have a Spearman correlation of around 0.99, further confirming their redundancy.

Weather and Time-related Features:
fresh_snow_24h:cm, fresh_snow_12h:cm, fresh_snow_6h:cm, fresh_snow_3h:cm, and fresh_snow_1h:cm: These features show high Spearman correlations ranging from 0.75 to 0.94, indicating they are capturing similar snowfall patterns over different time frames.

solar_zenith_angle:d and sun_elevation:d: Their Spearman correlation is -0.99, which is in line with their natural inverse relationship.

Correlations with Target Variable (pv_measurement):
    clear_sky_energy_1h:J: This has a Spearman correlation of 0.78 with the target variable, emphasizing its importance for prediction.

sun_elevation:d: This feature also has a Spearman correlation of 0.76 with the target variable, underlining its significance.

Other Interesting Correlations:
air_density_2m:kgm3 and t_1000hPa:K: These features have a Spearman correlation of -0.91, suggesting a robust inverse relationship even when accounting for non-linearities.

## 3.4 Correlations Based on Dataset

## 3.5 First Feature Extraction Based on Feature Redundancy

# 4 In-depth Feature Information

## 4.1 Feature Extraction for Further EDA

In [ ]:
# Prepare the data
X = df.drop(columns=['pv_measurement', 'time'])  # Drop the index, target, and time columns
y = df['pv_measurement']  # Target variable

### 4.1.1 Using XGBoost

In [ ]:
# Initialize and fit the XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X, y)

# Extract feature importances
feature_importances = xgb_model.feature_importances_

# Pair feature names with their importance scores
feature_importance_dict = dict(zip(X.columns, feature_importances))

# Sort the features by importance
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Extract the top 20 most important features
top_20_features = sorted_features[:20]

# Plotting the feature importances
plt.figure(figsize=(12, 8))
plt.barh([x[0] for x in reversed(top_20_features)], [x[1] for x in reversed(top_20_features)])
plt.xlabel('Importance')
plt.title('Top 20 Most Important Features According to XGBoost')
plt.show()

### 4.1.2 Using Mutual Information

In [ ]:
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns)
mi_series.sort_values(ascending=False, inplace=True)

# Top 20 features based on Mutual Information
top_20_mi = mi_series[:20]

plt.figure(figsize=(12, 8))
plt.barh(top_20_mi.index, top_20_mi.values)
plt.xlabel('Mutual Information')
plt.title('Top 20 Most Important Features According to Mutual Information')
plt.show()

### 4.1.3 Using Recursive Feature Elimination

In [ ]:
estimator = xgb.XGBRegressor()
selector = RFE(estimator, n_features_to_select=20, step=1)
selector = selector.fit(X, y)

top_20_rfe = pd.Series(selector.support_, index=X.columns)
top_20_rfe = top_20_rfe[top_20_rfe].index

# Plotting the top 20 most important features according to RFE
top_20_rfe

### 4.1.4 Using F-Regression

In [ ]:
F_values, p_values = f_regression(X, y)
f_reg_series = pd.Series(F_values, index=X.columns)
f_reg_series.sort_values(ascending=False, inplace=True)

# Top 20 features based on F-Regression
top_20_f_reg = f_reg_series[:20]

plt.figure(figsize=(12, 8))
plt.barh(top_20_f_reg.index, top_20_f_reg.values)
plt.xlabel('F-Value')
plt.title('Top 20 Most Important Features According to F-Regression')
plt.show()

## 4.2 Seasonal Decomposition

## 4.3 Stationarity 

## 4.4 Autocorrelation (ACF/PACF)

# 5. Feature Engineering

## 5.1 Time based Features

In [ ]:
def date_features(df):
    df['hour'] = pd.to_datetime(df['time']).dt.hour
    df['day'] = pd.to_datetime(df['time']).dt.dayofyear
    df['month'] = pd.to_datetime(df['time']).dt.month
    df['quarter'] = pd.to_datetime(df['time']).dt.quarter

    df['lagged_pv_measurement_1h'] = df['pv_measurement'].shift(1)
    df['lagged_pv_measurement_3h'] = df['pv_measurement'].shift(3)
    df['lagged_pv_measurement_6h'] = df['pv_measurement'].shift(6)

    df['rolling_mean_pv_measurement_3h'] = df['pv_measurement'].rolling(window=3).mean()
    return df

date_features(df)

In [ ]:
# Make sure to have a column 'id' for each time series and 'time' for the time stamps
# extracted_features = extract_features(df, column_id='id', column_sort='time')